In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import nltk
import re
import string
import os
import wordninja
import json
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import reuters
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
TRAIN_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/train"
TEST_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/test"

def read_text_file(path):
    labels = ['neg','pos']
    if os.path.exists(path):
        text=[]
        text_label =[]
        for directory_name in os.listdir(path):
            if directory_name in labels:
                label_index = labels.index(directory_name)
                data_path = os.path.join(path,directory_name)
                for file in os.listdir(data_path):
                    with open(os.path.join(data_path,file),'r', encoding='utf-8') as f:
                        text.append(f.read())
                        text_label.append(label_index)
        return pd.DataFrame(text,columns =['texts']),pd.DataFrame(text_label,columns =['label'])
    
x_train,y_train = read_text_file(TRAIN_DATA_PATH) 
x_test,y_test = read_text_file(TEST_DATA_PATH) 

In [6]:
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

In [7]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

train[['texts']] = train[['texts']].applymap(lambda x:remove_punctuation(x))
train[['texts']] = train[['texts']].applymap(lambda x:x.lower())
test[['texts']] = test[['texts']].applymap(lambda x:remove_punctuation(x))
test[['texts']] = test[['texts']].applymap(lambda x:x.lower())

In [8]:
def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

In [9]:
X_train = train.drop(columns=['label'])
X_test = test.drop(columns=['label'])
y_train = train['label']
y_test = test['label']

X_train = X_train['texts'].apply(clean_text)
X_test = X_test['texts'].apply(clean_text)

pattern = '[^a-z ]'
Clean_X_train=[]
Clean_X_test=[]

for sen in X_train:
    Clean_X_train.append(re.sub(pattern, '', str(sen)))
    
for sen in X_test:
    Clean_X_test.append(re.sub(pattern, '', str(sen)))

In [10]:
y_train=list(y_train)
y_test=list(y_test)

In [11]:
train_df = pd.DataFrame({'X_train': Clean_X_train, 'y_train': y_train})
test_df = pd.DataFrame({'X_test': Clean_X_test, 'y_test': y_test})

In [12]:
# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 8000개의 샘플을 랜덤하게 추출
train_df = train_df.groupby('y_train').apply(lambda x: x.sample(n=8000, random_state=42))

# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 2000개의 샘플을 랜덤하게 추출
test_df = test_df.groupby('y_test').apply(lambda x: x.sample(n=2000, random_state=42))

# 인덱스를 재설정합니다. drop=True 옵션을 사용하여 기존 인덱스를 제거합니다.
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [13]:
x_train = train_df['X_train'].tolist()
y_train = train_df['y_train'].tolist()
x_test = test_df['X_test'].tolist()
y_test = test_df['y_test'].tolist()

In [14]:
to_txt=x_train+x_test
y=y_train+y_test

In [15]:
#문서 길이를 조정하는 코드 텍스트 리스트와 최대 길이를 입력하면 단어 시퀀스를 최대 길이 이내로 truncate 해줌
def limit_words(text_list, max_words):
    new_list = []
    for text in text_list:
        words = text.split()
        if len(words) > max_words:
            words = words[:max_words]
        new_text = ' '.join(words)
        new_list.append(new_text)
    return new_list

In [16]:
#사전학습된 버트는 최대 임베딩 토큰 수(512)가 정해져 있으므로 넉넉하게 최대길이를 300으로 제한
to_txt=limit_words(to_txt,300)

In [17]:
to_txt

['the old axiom that bored people are boring people is well demonstrated in women in love the script taken from d h lawrences novel contains an endless flow of concepts that are at best sophomoricbr br what a pity so much effort went into so vacuous an exercise what an empty array of characters given such attention in spite of high production values this film comes across as tedious as its personnelbr br a revisit in merely confirms a impression of juvenille minds in adult bodies dawdling nowhere and fumbling every step of the way',
 'nothing in this movie is funny i thought the premise giving a human the libido of a randy ram was interesting and should provide for some laughs wrong there is simply nothing funny about the movie for example the main character making a pass at a goat in heat in the middle of a farmers yard is not funny it borders on obscenity they are toying around with bestiality in this film on one level and it just aint funnybr br we all know that dogs will eat anythi

버트 모델 정의

In [18]:
import torch
import numpy as np
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from itertools import groupby
from operator import itemgetter


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 미리 학습된 모델(가중치) 불러오기
model = BertModel.from_pretrained('bert-base-uncased')

In [19]:
text = to_txt[5]


marked_text = "[CLS] " + text + " [SEP]"

tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화

indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids = [1] * len(tokenized_text)

# Python list를 PyTorch tensor로 변환하기 
tokens_tensor = torch.tensor([indexed_text])
segments_tensors = torch.tensor([segments_ids])

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors )
    
token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])

token_vecs_sum = []

for token in token_embeddings:
    
    sum_vec = torch.sum(token[-4:], dim=0)
    
    token_vecs_sum.append(np.array(sum_vec))
    

token_vecs_sum=np.array(token_vecs_sum)

In [20]:
def split_continuous_integers(lst):                                      #연속된 정수리스트를 split 해주는 함수
    for k, g in groupby(enumerate(lst), lambda i_x:i_x[0]-i_x[1]):
        yield list(map(itemgetter(1), g))
        
def add_previous_number(lst):                                            #최초 서브토큰 인덱스 추가
    return [[sub_lst[0] - 1] + sub_lst for sub_lst in lst]

def bert_word_embedding(text):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text) #서브토큰화
    indexed_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # Python list를 PyTorch tensor로 변환하기 
    tokens_tensor = torch.tensor([indexed_text])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors )
    token_embeddings = torch.stack(encoded_layers, dim=0) #12개 레이어 쌓기 # torch.Size([12, 1, 22, 768])
    token_embeddings = torch.squeeze(token_embeddings, dim=1) #배치 차원 제거 # torch.Size([12, 22, 768])
    token_embeddings = token_embeddings.permute(1,0,2) #위치 변환 # torch.Size([22, 12, 768])
    
    token_vecs_sum = []
    for token in token_embeddings:                #인코더의 마지막 4개의 은닉 상태를 합쳐 최종 벡터(4개 합친 것이 성능이 가장 좋음)
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(np.array(sum_vec))
    
    token_vecs_sum=np.array(token_vecs_sum)
    #서브토큰을 결합해 단어에 대한 임베딩 벡터를 얻자
    subword_indices = [i for i, token in enumerate(tokenized_text) if '##' in token]
    index_list = add_previous_number(list(split_continuous_integers(subword_indices))) #index_list는 서브토큰에 해당하는 인덱스를 한 리스트에 묶어줌
    
    new_token_vecs_sum = []
    last_index = 0

    for subword_inx_list in index_list:
        # 이전 인덱스와 현재 인덱스 그룹 사이의 벡터를 추가합니다.
        new_token_vecs_sum.extend(token_vecs_sum[last_index:subword_inx_list[0]])

        # 현재 인덱스 그룹에 해당하는 벡터의 평균을 계산하고 추가합니다.
        avg_vecs = np.mean(token_vecs_sum[subword_inx_list], axis=0)
        new_token_vecs_sum.append(avg_vecs)

        # 마지막 인덱스를 업데이트합니다.
        last_index = subword_inx_list[-1] + 1

    # 마지막 인덱스 이후의 벡터를 추가합니다.
    new_token_vecs_sum.extend(token_vecs_sum[last_index:])

    # 결과를 numpy array로 변환합니다.
    new_token_vecs_sum = np.array(new_token_vecs_sum[1:-1])
    
    return new_token_vecs_sum

In [21]:
token_vecs_sum=bert_word_embedding(text)
token_vecs_sum.shape

(108, 768)

임베딩

In [22]:
x_train=to_txt[:16000]
x_test=to_txt[16000:]

In [23]:
except_lst=[]
for i in range(len(x_train)):
    try:
        text = to_txt[i]
        embed_matrix=bert_word_embedding(text)       
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(95, 768)
(213, 768)
(58, 768)
(247, 768)
(300, 768)
(108, 768)
(300, 768)
(300, 768)
(145, 768)
(228, 768)
(120, 768)
(46, 768)
(140, 768)
(59, 768)
(161, 768)
(103, 768)
(288, 768)
(173, 768)
(300, 768)
(223, 768)
(300, 768)
(300, 768)
(176, 768)
(106, 768)
(300, 768)
(300, 768)
(278, 768)
(184, 768)
(126, 768)
(95, 768)
(136, 768)
(148, 768)
(140, 768)
(126, 768)
(169, 768)
(222, 768)
(300, 768)
(293, 768)
(82, 768)
(169, 768)
(300, 768)
(300, 768)
(128, 768)
(300, 768)
(134, 768)
(139, 768)
(148, 768)
(171, 768)
(245, 768)
(97, 768)
(170, 768)
(131, 768)
(300, 768)
(121, 768)
(129, 768)
(57, 768)
(116, 768)
(85, 768)
(300, 768)
(193, 768)
(188, 768)
(130, 768)
(300, 768)
(146, 768)
(300, 768)
(300, 768)
(288, 768)
(300, 768)
(118, 768)
(150, 768)
(300, 768)
(300, 768)
(240, 768)
(130, 768)
(300, 768)
(160, 768)
(113, 768)
(127, 768)
(248, 768)
(300, 768)
(169, 768)
(153, 768)
(132, 768)
(117, 768)
(300, 768)
(169, 768)
(192, 768)
(158, 768)
(162, 768)
(208, 768)
(201, 768)
(140, 76

(133, 768)
(300, 768)
(300, 768)
(262, 768)
(79, 768)
(138, 768)
(296, 768)
(217, 768)
(208, 768)
(159, 768)
(300, 768)
(300, 768)
(253, 768)
(193, 768)
(286, 768)
(300, 768)
(300, 768)
(162, 768)
(272, 768)
(281, 768)
(300, 768)
(220, 768)
(256, 768)
(300, 768)
(135, 768)
(44, 768)
(300, 768)
(72, 768)
(300, 768)
(225, 768)
(175, 768)
(58, 768)
(255, 768)
(125, 768)
(288, 768)
(189, 768)
(300, 768)
(132, 768)
(164, 768)
(114, 768)
(252, 768)
(187, 768)
(293, 768)
(174, 768)
(274, 768)
(151, 768)
(122, 768)
(148, 768)
(178, 768)
(182, 768)
(181, 768)
(160, 768)
(300, 768)
(300, 768)
(300, 768)
(176, 768)
(300, 768)
(154, 768)
(150, 768)
(132, 768)
(300, 768)
(300, 768)
(300, 768)
(150, 768)
(150, 768)
(300, 768)
(131, 768)
(300, 768)
(112, 768)
(130, 768)
(258, 768)
(300, 768)
(300, 768)
(300, 768)
(153, 768)
(142, 768)
(300, 768)
(179, 768)
(150, 768)
(300, 768)
(136, 768)
(113, 768)
(273, 768)
(278, 768)
(300, 768)
(121, 768)
(172, 768)
(171, 768)
(177, 768)
(151, 768)
(78, 768)
(241

(300, 768)
(131, 768)
(69, 768)
(107, 768)
(132, 768)
(132, 768)
(300, 768)
(179, 768)
(300, 768)
(300, 768)
(256, 768)
(50, 768)
(46, 768)
(300, 768)
(68, 768)
(91, 768)
(169, 768)
(139, 768)
(118, 768)
(300, 768)
(114, 768)
(93, 768)
(70, 768)
(218, 768)
(120, 768)
(300, 768)
(300, 768)
(143, 768)
(90, 768)
(300, 768)
(216, 768)
(50, 768)
(300, 768)
(133, 768)
(142, 768)
(205, 768)
(48, 768)
(120, 768)
(158, 768)
(300, 768)
(249, 768)
(164, 768)
(98, 768)
(235, 768)
(300, 768)
(102, 768)
(110, 768)
(184, 768)
(300, 768)
(300, 768)
(141, 768)
(133, 768)
(216, 768)
(171, 768)
(300, 768)
(39, 768)
(149, 768)
(300, 768)
(291, 768)
(123, 768)
(259, 768)
(97, 768)
(146, 768)
(300, 768)
(149, 768)
(102, 768)
(124, 768)
(112, 768)
(163, 768)
(143, 768)
(158, 768)
(270, 768)
(201, 768)
(222, 768)
(212, 768)
(71, 768)
(210, 768)
(58, 768)
(300, 768)
(300, 768)
(300, 768)
(270, 768)
(62, 768)
(300, 768)
(109, 768)
(208, 768)
(145, 768)
(300, 768)
(140, 768)
(122, 768)
(110, 768)
(130, 768)
(300

(69, 768)
(300, 768)
(86, 768)
(100, 768)
(74, 768)
(300, 768)
(111, 768)
(104, 768)
(300, 768)
(195, 768)
(131, 768)
(214, 768)
(131, 768)
(151, 768)
(300, 768)
(300, 768)
(154, 768)
(117, 768)
(148, 768)
(300, 768)
(250, 768)
(194, 768)
(235, 768)
(135, 768)
(152, 768)
(300, 768)
(152, 768)
(103, 768)
(198, 768)
(300, 768)
(263, 768)
(41, 768)
(177, 768)
(300, 768)
(75, 768)
(67, 768)
(300, 768)
(132, 768)
(90, 768)
(125, 768)
(300, 768)
(27, 768)
(150, 768)
(101, 768)
(300, 768)
(67, 768)
(234, 768)
(187, 768)
(300, 768)
(133, 768)
(132, 768)
(159, 768)
(87, 768)
(223, 768)
(145, 768)
(265, 768)
(300, 768)
(96, 768)
(150, 768)
(165, 768)
(291, 768)
(191, 768)
(133, 768)
(143, 768)
(50, 768)
(265, 768)
(94, 768)
(68, 768)
(300, 768)
(122, 768)
(217, 768)
(300, 768)
(142, 768)
(300, 768)
(300, 768)
(132, 768)
(164, 768)
(118, 768)
(113, 768)
(123, 768)
(56, 768)
(123, 768)
(228, 768)
(212, 768)
(178, 768)
(112, 768)
(247, 768)
(70, 768)
(178, 768)
(126, 768)
(239, 768)
(160, 768)
(130

(200, 768)
(242, 768)
(133, 768)
(135, 768)
(140, 768)
(300, 768)
(190, 768)
(300, 768)
(145, 768)
(165, 768)
(128, 768)
(300, 768)
(72, 768)
(300, 768)
(147, 768)
(64, 768)
(237, 768)
(211, 768)
(101, 768)
(132, 768)
(143, 768)
(234, 768)
(300, 768)
(300, 768)
(188, 768)
(125, 768)
(289, 768)
(257, 768)
(300, 768)
(234, 768)
(149, 768)
(185, 768)
(231, 768)
(300, 768)
(128, 768)
(139, 768)
(193, 768)
(124, 768)
(167, 768)
(300, 768)
(123, 768)
(107, 768)
(300, 768)
(122, 768)
(300, 768)
(160, 768)
(112, 768)
(90, 768)
(69, 768)
(110, 768)
(156, 768)
(143, 768)
(300, 768)
(186, 768)
(62, 768)
(151, 768)
(247, 768)
(129, 768)
(300, 768)
(286, 768)
(300, 768)
(132, 768)
(284, 768)
(114, 768)
(300, 768)
(300, 768)
(246, 768)
(109, 768)
(92, 768)
(184, 768)
(154, 768)
(45, 768)
(124, 768)
(300, 768)
(158, 768)
(139, 768)
(300, 768)
(213, 768)
(250, 768)
(300, 768)
(125, 768)
(154, 768)
(300, 768)
(116, 768)
(256, 768)
(300, 768)
(143, 768)
(171, 768)
(269, 768)
(173, 768)
(153, 768)
(51, 7

...

(162, 768)
(82, 768)
(300, 768)
(131, 768)
(189, 768)
(177, 768)
(219, 768)
(32, 768)
(300, 768)
(114, 768)
(207, 768)
(300, 768)
(159, 768)
(135, 768)
(185, 768)
(300, 768)
(250, 768)
(145, 768)
(300, 768)
(119, 768)
(148, 768)
(58, 768)
(135, 768)
(66, 768)
(300, 768)
(44, 768)
(82, 768)
(176, 768)
(130, 768)
(141, 768)
(300, 768)
(121, 768)
(148, 768)
(300, 768)
(35, 768)
(104, 768)
(157, 768)
(123, 768)
(60, 768)
(300, 768)
(300, 768)
(160, 768)
(131, 768)
(151, 768)
(206, 768)
(48, 768)
(300, 768)
(263, 768)
(231, 768)
(198, 768)
(265, 768)
(57, 768)
(179, 768)
(167, 768)
(255, 768)
(134, 768)
(207, 768)
(175, 768)
(136, 768)
(96, 768)
(68, 768)
(172, 768)
(146, 768)
(175, 768)
(300, 768)
(300, 768)
(200, 768)
(218, 768)
(252, 768)
(284, 768)
(246, 768)
(112, 768)
(232, 768)
(300, 768)
(240, 768)
(190, 768)
(165, 768)
(218, 768)
(149, 768)
(149, 768)
(177, 768)
(153, 768)
(300, 768)
(300, 768)
(142, 768)
(131, 768)
(300, 768)
(55, 768)
(300, 768)
(134, 768)
(85, 768)
(100, 768)
(2

(100, 768)
(40, 768)
(77, 768)
(29, 768)
(78, 768)
(98, 768)
(300, 768)
(125, 768)
(235, 768)
(270, 768)
(85, 768)
(300, 768)
(300, 768)
(59, 768)
(61, 768)
(118, 768)
(207, 768)
(246, 768)
(194, 768)
(97, 768)
(152, 768)
(126, 768)
(42, 768)
(258, 768)
(84, 768)
(163, 768)
(215, 768)
(161, 768)
(124, 768)
(184, 768)
(212, 768)
(153, 768)
(34, 768)
(105, 768)
(300, 768)
(122, 768)
(115, 768)
(97, 768)
(138, 768)
(300, 768)
(121, 768)
(170, 768)
(90, 768)
(175, 768)
(133, 768)
(120, 768)
(110, 768)
(197, 768)
(179, 768)
(110, 768)
(124, 768)
(199, 768)
(211, 768)
(226, 768)
(300, 768)
(184, 768)
(126, 768)
(166, 768)
(117, 768)
(210, 768)
(114, 768)
(160, 768)
(52, 768)
(300, 768)
(300, 768)
(217, 768)
(54, 768)
(149, 768)
(219, 768)
(129, 768)
(78, 768)
(49, 768)
(187, 768)
(65, 768)
(250, 768)
(243, 768)
(225, 768)
(141, 768)
(300, 768)
(300, 768)
(300, 768)
(55, 768)
(69, 768)
(126, 768)
(158, 768)
(119, 768)
(122, 768)
(300, 768)
(188, 768)
(181, 768)
(271, 768)
(89, 768)
(187, 768)

(108, 768)
(73, 768)
(259, 768)
(300, 768)
(226, 768)
(124, 768)
(241, 768)
(300, 768)
(264, 768)
(247, 768)
(100, 768)
(70, 768)
(185, 768)
(173, 768)
(115, 768)
(127, 768)
(278, 768)
(300, 768)
(300, 768)
(200, 768)
(300, 768)
(171, 768)
(300, 768)
(138, 768)
(118, 768)
(247, 768)
(212, 768)
(300, 768)
(300, 768)
(193, 768)
(116, 768)
(215, 768)
(182, 768)
(300, 768)
(250, 768)
(300, 768)
(178, 768)
(91, 768)
(300, 768)
(127, 768)
(300, 768)
(300, 768)
(120, 768)
(33, 768)
(68, 768)
(300, 768)
(90, 768)
(54, 768)
(182, 768)
(120, 768)
(161, 768)
(191, 768)
(176, 768)
(140, 768)
(178, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(286, 768)
(130, 768)
(61, 768)
(105, 768)
(137, 768)
(300, 768)
(151, 768)
(183, 768)
(261, 768)
(116, 768)
(197, 768)
(78, 768)
(48, 768)
(107, 768)
(53, 768)
(300, 768)
(152, 768)
(300, 768)
(300, 768)
(180, 768)
(163, 768)
(124, 768)
(190, 768)
(161, 768)
(170, 768)
(99, 768)
(206, 768)
(44, 768)
(180, 768)
(104, 768)
(185, 768)
(300, 768)
(

(157, 768)
(153, 768)
(300, 768)
(300, 768)
(180, 768)
(300, 768)
(300, 768)
(300, 768)
(298, 768)
(230, 768)
(159, 768)
(235, 768)
(300, 768)
(279, 768)
(208, 768)
(223, 768)
(92, 768)
(137, 768)
(124, 768)
(206, 768)
(300, 768)
(168, 768)
(300, 768)
(246, 768)
(152, 768)
(55, 768)
(129, 768)
(300, 768)
(300, 768)
(300, 768)
(110, 768)
(127, 768)
(187, 768)
(300, 768)
(177, 768)
(137, 768)
(300, 768)
(300, 768)
(201, 768)
(266, 768)
(273, 768)
(107, 768)
(300, 768)
(92, 768)
(145, 768)
(146, 768)
(113, 768)
(116, 768)
(107, 768)
(165, 768)
(118, 768)
(259, 768)
(300, 768)
(157, 768)
(185, 768)
(135, 768)
(300, 768)
(126, 768)
(158, 768)
(300, 768)
(300, 768)
(255, 768)
(300, 768)
(210, 768)
(120, 768)
(119, 768)
(188, 768)
(227, 768)
(300, 768)
(243, 768)
(300, 768)
(177, 768)
(41, 768)
(300, 768)
(261, 768)
(137, 768)
(135, 768)
(300, 768)
(189, 768)
(70, 768)
(189, 768)
(300, 768)
(132, 768)
(187, 768)
(283, 768)
(49, 768)
(51, 768)
(139, 768)
(33, 768)
(136, 768)
(300, 768)
(296, 7

(100, 768)
(124, 768)
(120, 768)
(201, 768)
(213, 768)
(300, 768)
(300, 768)
(208, 768)
(300, 768)
(138, 768)
(173, 768)
(47, 768)
(300, 768)
(300, 768)
(62, 768)
(114, 768)
(145, 768)
(193, 768)
(114, 768)
(185, 768)
(248, 768)
(278, 768)
(161, 768)
(163, 768)
(250, 768)
(98, 768)
(300, 768)
(132, 768)
(128, 768)
(247, 768)
(176, 768)
(300, 768)
(182, 768)
(300, 768)
(165, 768)
(300, 768)
(138, 768)
(141, 768)
(171, 768)
(129, 768)
(198, 768)
(53, 768)
(184, 768)
(138, 768)
(113, 768)
(124, 768)
(228, 768)
(93, 768)
(134, 768)
(116, 768)
(159, 768)
(116, 768)
(180, 768)
(118, 768)
(246, 768)
(300, 768)
(98, 768)
(300, 768)
(300, 768)
(175, 768)
(235, 768)
(134, 768)
(231, 768)
(128, 768)
(195, 768)
(100, 768)
(300, 768)
(133, 768)
(138, 768)
(199, 768)
(182, 768)
(300, 768)
(146, 768)
(241, 768)
(195, 768)
(115, 768)
(79, 768)
(110, 768)
(134, 768)
(290, 768)
(240, 768)
(300, 768)
(167, 768)
(124, 768)
(300, 768)
(140, 768)
(145, 768)
(140, 768)
(76, 768)
(130, 768)
(300, 768)
(156, 7

In [24]:
#제외된 문서가 존재할 경우 아래 셀에 있는 코드의 주석을 풀고 입력 시퀀스의 길이를 줄인 후 재시도

print('제외된 문서의 수: ',len(except_lst))

with open('except_lst.pkl', 'wb') as file:
    pickle.dump(except_lst, file)

print("except_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_lst를 피클 파일로 저장했습니다.


In [15]:
# new_except_lst=[]
# to_txt_long_train=[]
# for idx in except_lst:
#     to_txt_long_train.append(to_txt[idx])
    
# to_txt_long_train=limit_words(to_txt_long_train,200)

# for i in except_lst:
#     try:
#         cnt=0
#         text = to_txt_long_train[cnt]
#         cnt+=1
#         embed_matrix=bert_word_embedding(text)   
#         np.save('./bert_embedding/train(bert)/doc%d'%i,embed_matrix)
#         print(embed_matrix.shape)
#     except:
#         new_except_lst.append(i)   #단어를 -개로 제한했는데도 sub_token 길이가 512가 넘는 문서는 new_except_lst에 추가

(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)
(200, 768)


In [25]:
new_except_lst

NameError: name 'new_except_lst' is not defined

In [26]:
except_test_lst=[]

for i in range(len(x_test)):
    try:
        text = to_txt[i+16000]
        embed_matrix=bert_word_embedding(text)
        #임베딩된 문서 임베딩 행렬을 저장
        np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
        print(embed_matrix.shape)
    except:
        except_test_lst.append(i)   #sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(290, 768)
(157, 768)
(282, 768)
(149, 768)
(204, 768)
(300, 768)
(245, 768)
(135, 768)
(104, 768)
(159, 768)
(300, 768)
(94, 768)
(300, 768)
(300, 768)
(116, 768)
(246, 768)
(135, 768)
(300, 768)
(141, 768)
(115, 768)
(105, 768)
(120, 768)
(300, 768)
(300, 768)
(170, 768)
(244, 768)
(153, 768)
(149, 768)
(216, 768)
(300, 768)
(229, 768)
(229, 768)
(256, 768)
(130, 768)
(113, 768)
(123, 768)
(300, 768)
(166, 768)
(157, 768)
(300, 768)
(33, 768)
(60, 768)
(182, 768)
(172, 768)
(130, 768)
(283, 768)
(106, 768)
(163, 768)
(300, 768)
(122, 768)
(177, 768)
(300, 768)
(131, 768)
(121, 768)
(124, 768)
(165, 768)
(143, 768)
(165, 768)
(58, 768)
(300, 768)
(130, 768)
(300, 768)
(135, 768)
(111, 768)
(122, 768)
(300, 768)
(115, 768)
(40, 768)
(135, 768)
(98, 768)
(152, 768)
(300, 768)
(219, 768)
(200, 768)
(166, 768)
(177, 768)
(36, 768)
(84, 768)
(120, 768)
(206, 768)
(158, 768)
(131, 768)
(141, 768)
(296, 768)
(133, 768)
(172, 768)
(300, 768)
(261, 768)
(51, 768)
(170, 768)
(300, 768)
(97, 768

(300, 768)
(193, 768)
(176, 768)
(131, 768)
(172, 768)
(183, 768)
(300, 768)
(80, 768)
(232, 768)
(114, 768)
(300, 768)
(245, 768)
(267, 768)
(300, 768)
(269, 768)
(139, 768)
(53, 768)
(33, 768)
(106, 768)
(299, 768)
(155, 768)
(96, 768)
(139, 768)
(123, 768)
(300, 768)
(167, 768)
(214, 768)
(300, 768)
(300, 768)
(210, 768)
(126, 768)
(300, 768)
(139, 768)
(149, 768)
(300, 768)
(125, 768)
(128, 768)
(150, 768)
(196, 768)
(70, 768)
(300, 768)
(195, 768)
(221, 768)
(154, 768)
(31, 768)
(300, 768)
(300, 768)
(177, 768)
(189, 768)
(300, 768)
(300, 768)
(112, 768)
(121, 768)
(158, 768)
(213, 768)
(60, 768)
(186, 768)
(150, 768)
(168, 768)
(300, 768)
(63, 768)
(300, 768)
(113, 768)
(272, 768)
(163, 768)
(124, 768)
(180, 768)
(300, 768)
(137, 768)
(148, 768)
(170, 768)
(201, 768)
(155, 768)
(195, 768)
(300, 768)
(300, 768)
(300, 768)
(153, 768)
(215, 768)
(300, 768)
(247, 768)
(166, 768)
(300, 768)
(132, 768)
(300, 768)
(158, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(106, 768)
(300, 7

(300, 768)
(300, 768)
(114, 768)
(300, 768)
(142, 768)
(118, 768)
(300, 768)
(82, 768)
(300, 768)
(288, 768)
(167, 768)
(106, 768)
(219, 768)
(86, 768)
(175, 768)
(300, 768)
(88, 768)
(58, 768)
(155, 768)
(118, 768)
(300, 768)
(157, 768)
(272, 768)
(273, 768)
(254, 768)
(300, 768)
(110, 768)
(49, 768)
(223, 768)
(272, 768)
(111, 768)
(153, 768)
(64, 768)
(300, 768)
(134, 768)
(179, 768)
(138, 768)
(147, 768)
(300, 768)
(220, 768)
(108, 768)
(174, 768)
(65, 768)
(177, 768)
(164, 768)
(131, 768)
(126, 768)
(300, 768)
(44, 768)
(169, 768)
(300, 768)
(219, 768)
(161, 768)
(300, 768)
(294, 768)
(121, 768)
(178, 768)
(122, 768)
(143, 768)
(300, 768)
(152, 768)
(300, 768)
(268, 768)
(219, 768)
(287, 768)
(87, 768)
(161, 768)
(162, 768)
(157, 768)
(155, 768)
(218, 768)
(200, 768)
(25, 768)
(153, 768)
(300, 768)
(134, 768)
(300, 768)
(138, 768)
(143, 768)
(121, 768)
(233, 768)
(288, 768)
(163, 768)
(271, 768)
(136, 768)
(106, 768)
(127, 768)
(212, 768)
(139, 768)
(62, 768)
(182, 768)
(300, 768)

(189, 768)
(185, 768)
(300, 768)
(213, 768)
(151, 768)
(115, 768)
(167, 768)
(213, 768)
(55, 768)
(300, 768)
(300, 768)
(115, 768)
(300, 768)
(290, 768)
(103, 768)
(110, 768)
(300, 768)
(300, 768)
(300, 768)
(300, 768)
(287, 768)
(126, 768)
(121, 768)
(51, 768)
(300, 768)
(300, 768)
(267, 768)
(128, 768)
(119, 768)
(54, 768)
(40, 768)
(126, 768)
(300, 768)
(200, 768)
(274, 768)
(300, 768)
(125, 768)
(300, 768)
(300, 768)
(300, 768)
(62, 768)
(300, 768)
(151, 768)
(279, 768)
(300, 768)
(280, 768)
(146, 768)
(210, 768)
(67, 768)
(226, 768)
(191, 768)
(134, 768)
(145, 768)
(300, 768)
(300, 768)
(132, 768)
(291, 768)
(76, 768)
(149, 768)
(228, 768)
(52, 768)
(235, 768)
(92, 768)
(60, 768)
(163, 768)
(121, 768)
(300, 768)
(231, 768)
(51, 768)
(300, 768)
(300, 768)
(108, 768)
(123, 768)
(116, 768)
(236, 768)
(143, 768)
(114, 768)
(300, 768)
(125, 768)
(65, 768)
(139, 768)
(151, 768)
(136, 768)
(292, 768)
(209, 768)
(113, 768)
(188, 768)
(142, 768)
(147, 768)
(63, 768)
(263, 768)
(300, 768)
(

(120, 768)
(144, 768)
(223, 768)
(129, 768)
(106, 768)
(83, 768)
(145, 768)
(130, 768)
(300, 768)
(180, 768)
(138, 768)
(123, 768)
(181, 768)
(133, 768)
(118, 768)
(236, 768)
(248, 768)
(139, 768)
(92, 768)
(157, 768)
(166, 768)
(78, 768)
(185, 768)
(110, 768)
(189, 768)
(300, 768)
(300, 768)
(300, 768)
(207, 768)
(300, 768)
(300, 768)
(98, 768)
(128, 768)
(118, 768)
(178, 768)
(300, 768)
(300, 768)
(134, 768)
(84, 768)
(29, 768)
(288, 768)
(211, 768)
(266, 768)
(137, 768)
(110, 768)
(164, 768)
(300, 768)
(66, 768)
(96, 768)
(144, 768)
(300, 768)
(287, 768)
(300, 768)
(90, 768)
(300, 768)
(112, 768)
(51, 768)
(100, 768)
(128, 768)
(300, 768)
(300, 768)
(218, 768)
(50, 768)
(300, 768)
(152, 768)
(164, 768)
(180, 768)
(179, 768)
(272, 768)
(133, 768)
(300, 768)
(126, 768)
(121, 768)
(300, 768)
(300, 768)
(284, 768)
(150, 768)
(154, 768)
(53, 768)
(166, 768)
(227, 768)
(270, 768)
(57, 768)
(130, 768)
(45, 768)
(300, 768)
(172, 768)
(62, 768)
(168, 768)
(300, 768)
(300, 768)
(149, 768)
(82

(224, 768)
(48, 768)
(178, 768)
(300, 768)
(174, 768)
(95, 768)
(129, 768)
(300, 768)
(131, 768)
(260, 768)
(180, 768)
(300, 768)
(55, 768)
(300, 768)
(94, 768)
(120, 768)
(226, 768)
(125, 768)
(300, 768)
(300, 768)
(149, 768)
(148, 768)
(199, 768)
(297, 768)
(161, 768)
(300, 768)
(153, 768)
(174, 768)
(219, 768)
(300, 768)
(121, 768)
(174, 768)
(195, 768)
(300, 768)
(300, 768)
(179, 768)
(176, 768)
(144, 768)
(90, 768)
(300, 768)
(120, 768)
(275, 768)
(300, 768)
(210, 768)
(124, 768)
(247, 768)
(300, 768)
(194, 768)
(183, 768)
(299, 768)
(49, 768)
(95, 768)
(60, 768)
(133, 768)
(37, 768)
(300, 768)
(116, 768)
(300, 768)
(182, 768)
(300, 768)
(153, 768)
(108, 768)
(54, 768)
(87, 768)
(300, 768)
(42, 768)
(233, 768)
(161, 768)
(300, 768)
(169, 768)
(79, 768)
(300, 768)
(121, 768)
(76, 768)
(112, 768)
(300, 768)
(67, 768)
(137, 768)
(44, 768)
(299, 768)
(300, 768)
(300, 768)
(249, 768)
(110, 768)
(219, 768)
(131, 768)
(284, 768)
(300, 768)
(177, 768)
(79, 768)
(267, 768)
(121, 768)
(72, 

In [27]:
#제외된 문서가 존재할 경우 아래 셀에 있는 코드의 주석을 풀고 입력 시퀀스의 길이를 줄인 후 재시도

print('제외된 문서의 수: ',len(except_test_lst))

with open('except_test_lst.pkl', 'wb') as file:
    pickle.dump(except_test_lst, file)

print("except_test_lst를 피클 파일로 저장했습니다.")

제외된 문서의 수:  0
except_test_lst를 피클 파일로 저장했습니다.


In [23]:
# new_except_test_lst=[]
# to_txt_long_test=[]
# for idx in except_test_lst:
#     to_txt_long_test.append(to_txt[idx+16000])
    
# to_txt_long_test=limit_words(to_txt_long_test,180)

# for i in except_test_lst:
#     try:
#         cnt=0
#         text = to_txt_long_test[cnt]
#         cnt+=1
#         embed_matrix=bert_word_embedding(text)       
#         np.save('./bert_embedding/test(bert)/test%d'%i,embed_matrix)
#         print(embed_matrix.shape)
#     except:
#         new_except_test_lst.append(i)   #단어를 -개로 제한했는데도 sub_token 길이가 512가 넘는 문서는 except_lst에 추가

(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)
(180, 768)


In [25]:
new_except_test_lst

[]